In [5]:
import pandas as pd
from datetime import datetime

In [15]:
# примеры данных 
a = pd.read_csv('C:/Users/Лиен/Desktop/Jupyter/sample_1.csv')
b = pd.read_csv('C:/Users/Лиен/Desktop/Jupyter/sample_2.csv')

print(a)
print()
print(b)

   id  user_id   timestamp  billing_period  billing_total_price_usd
0   0       26  1708104067              74                       95
1   1       40  1707585667              25                       46
2   2       48  1704993667              50                       99
3   3       15  1704475267              32                       15
4   4       38  1706030467              84                       83

   id  user_id   timestamp  billing_period  billing_total_price_usd
0  23        1  1706555483               6                       60


In [13]:
def process(df):
    #приведение столбца timestamp в формат datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
    # расчет стоимости подписки в день
    df["contribution"] = df["billing_total_price_usd"] / df["billing_period"]
    # добавление столбца с датой окончания подписки
    df["end_date"] = df["timestamp"] + pd.to_timedelta(df["billing_period"], unit='D')
    
    # задаем период, для которого рассчитывается финансовый вклад
    first_day = pd.Timestamp("2024-02-01")
    last_day = pd.Timestamp("2024-03-01")
    
    # фильтруем даты 
    df_feb = df[(df["timestamp"] <= last_day) & (df["end_date"] >= first_day)]
    df['days_in_feb'] = (df_feb[["timestamp", "end_date"]].max(axis=1).clip(lower=first_day, upper=last_day)-df_feb[["timestamp", "end_date"]].min(axis=1).clip(lower=first_day, upper=last_day)).dt.days
    
    # расчет финансового вклада по каждой подписке
    df['contribution_sum']=df.contribution*df.days_in_feb
    
    # расчет финансового вклада со всех подписок пользователя
    user_contributions = df.groupby("user_id")["contribution_sum"].sum().reset_index()
    top3_users = user_contributions.nlargest(3, "contribution_sum")
    contribution_sum = round(top3_users['contribution_sum'].sum(),2)
    return contribution_sum

print(process(df=a))
print(process(df=b))

121.03
30.0
